In [1]:
from bs4 import BeautifulSoup,Comment
import pandas as pd
import requests
import urllib.request as urllib2
import lxml
import html5lib
import math

In [44]:
nba_names = pd.read_csv('NBAPlayerNames2017.csv')
print(nba_names.BBRefName[301])
print(nba_names.LastInitial[301])
print(nba_names.PlayerName[301])

mbahalu01
m
Luc Mbah a Moute


In [47]:
test_name = nba_names.BBRefName[301]
test_initial = nba_names.LastInitial[301]
test_player_name = nba_names.PlayerName[301]
#r=requests.get('https://www.basketball-reference.com/players/(%s)/(%s).html' % (test_initial,test_name)
r=requests.get('https://www.basketball-reference.com/players/{}/{}.html'.format(test_initial,test_name))
#r = requests.get('https://www.basketball-reference.com/players/h/hardeja01.html')
html_text = r.text
soup = BeautifulSoup(html_text,'lxml')
#print(soup.prettify())

#### Check_if_Rookie function checks the size of the Per Game table
#### If the table only has 2 rows, then the player is a rookie, and will not be considered in the analysis

In [48]:
def Check_if_Rookie(table_id):
    table = soup.find(attrs={"id":table_id})
    
    dfs = pd.read_html(str(table))
    df = dfs[0]
    #print(df)
    #print(len(df.index))
    if len(df)==2:
        return False;
    else:
        return True;

#### Extract_DataFrame function will go and retrieve the data from BasketballReference and put in into a dataframe
##### The Per Game table requires special handling, which is reflected in the if statement of the code.

In [49]:
def Extract_DataFrame(table_id):
    table = soup.find(attrs={"id":table_id})
    
    if table_id == 'all_per_game':
        dfs = pd.read_html(str(table))
        df = dfs[0]
        df.set_index('Season')

        for column in df:
            if not df[column].name.find("Unnamed"):
                del df[column]
    else:
        advanced_table = table.find(string=lambda text:isinstance(text,Comment))
        dfs = pd.read_html(str(advanced_table))
        df = dfs[0]
        df.set_index('Season')

        for column in df:
            if not df[column].name.find("Unnamed"):
                del df[column]
    
    return df;

In [91]:
bIsNotRookie = Check_if_Rookie(table_id = "all_per_game")
#print(bIsNotRookie)
if bIsNotRookie==True:
    df_per_game = Extract_DataFrame(table_id = "all_per_game")

    df_advanced = Extract_DataFrame(table_id = "all_advanced")

    df_salary = Extract_DataFrame(table_id = "all_all_salaries")

    df_final = pd.merge(df_per_game,df_advanced,how='inner',on=['Season','Age','Tm','Lg','Pos','G'])
    df_final = pd.merge(df_final,df_salary,how='inner', on=['Season','Lg'])
    df_final['Player'] = test_player_name
    df_final

In [88]:
df_final

,Season,Age,Tm,Lg,Pos,G,GS,MP_x,FG,FGA,...,WS,WS/48,.1,OBPM,DBPM,BPM,VORP,Team,Salary,Player
0,2008-09,22.0,MIL,NBA,PF,82,52,25.8,2.8,6.0,...,4.0,0.092,NaN,-1.9,2.2,0.2,1.2,Milwaukee Bucks,"$757,000",Luc Mbah a Moute
1,2009-10,23.0,MIL,NBA,PF,73,62,25.6,2.4,5.1,...,4.7,0.122,NaN,-1.6,2.9,1.4,1.6,Milwaukee Bucks,"$736,420",Luc Mbah a Moute
2,2010-11,24.0,MIL,NBA,PF,79,52,26.5,2.5,5.4,...,4.8,0.109,NaN,-2.3,2.2,0.0,1.0,Milwaukee Bucks,"$854,389",Luc Mbah a Moute
3,2011-12,25.0,MIL,NBA,PF,43,22,23.5,3.1,6.1,...,2.4,0.112,NaN,-1.4,0.6,-0.8,0.3,Milwaukee Bucks,"$5,000,000",Luc Mbah a Moute
4,2012-13,26.0,MIL,NBA,SF,58,45,22.9,2.6,6.5,...,0.5,0.017,NaN,-4.0,0.0,-4.1,-0.7,Milwaukee Bucks,"$4,795,192",Luc Mbah a Moute
5,2013-14,27.0,TOT,NBA,PF-SF,64,7,15.7,1.4,3.0,...,1.0,0.049,NaN,-3.0,0.6,-2.4,-0.1,Sacramento Kings,"$4,588,384",Luc Mbah a Moute
6,2013-14,27.0,SAC,NBA,SF,9,5,21.8,1.7,3.6,...,0.4,0.106,NaN,-0.9,2.1,1.1,0.2,Sacramento Kings,"$4,588,384",Luc Mbah a Moute
7,2013-14,27.0,MIN,NBA,PF,55,2,14.7,1.3,2.9,...,0.6,0.035,NaN,-3.6,0.3,-3.3,-0.3,Sacramento Kings,"$4,588,384",Luc Mbah a Moute
8,2014-15,28.0,PHI,NBA,PF,67,61,28.6,3.7,9.5,...,1.1,0.027,NaN,-3.2,0.1,-3.2,-0.6,Philadelphia 76ers,"$4,382,576",Luc Mbah a Moute
9,2015-16,29.0,LAC,NBA,PF,75,61,17.0,1.3,2.8,...,1.6,0.059,NaN,-3.0,1.5,-1.6,0.1,Los Angeles Clippers,"$1,270,964",Luc Mbah a Moute


In [92]:
#Need to change this so that it can handle players who have been traded multiple times midseason
df_final_TOT_season = df_final.loc[df_final['Tm']=='TOT']['Season']
df_final_TOT_season
df_final_TOT = df_final.loc[(df_final['Season']=='2013-14') & (df_final['Tm'] != 'TOT')] #Need to change this to use a variable
df_final_TOT
df_final_dropped = df_final.drop(df_final_TOT.index,0)
df_final_dropped

,Season,Age,Tm,Lg,Pos,G,GS,MP_x,FG,FGA,...,WS,WS/48,.1,OBPM,DBPM,BPM,VORP,Team,Salary,Player
0,2008-09,22.0,MIL,NBA,PF,82,52,25.8,2.8,6.0,...,4.0,0.092,NaN,-1.9,2.2,0.2,1.2,Milwaukee Bucks,"$757,000",Luc Mbah a Moute
1,2009-10,23.0,MIL,NBA,PF,73,62,25.6,2.4,5.1,...,4.7,0.122,NaN,-1.6,2.9,1.4,1.6,Milwaukee Bucks,"$736,420",Luc Mbah a Moute
2,2010-11,24.0,MIL,NBA,PF,79,52,26.5,2.5,5.4,...,4.8,0.109,NaN,-2.3,2.2,0.0,1.0,Milwaukee Bucks,"$854,389",Luc Mbah a Moute
3,2011-12,25.0,MIL,NBA,PF,43,22,23.5,3.1,6.1,...,2.4,0.112,NaN,-1.4,0.6,-0.8,0.3,Milwaukee Bucks,"$5,000,000",Luc Mbah a Moute
4,2012-13,26.0,MIL,NBA,SF,58,45,22.9,2.6,6.5,...,0.5,0.017,NaN,-4.0,0.0,-4.1,-0.7,Milwaukee Bucks,"$4,795,192",Luc Mbah a Moute
5,2013-14,27.0,TOT,NBA,PF-SF,64,7,15.7,1.4,3.0,...,1.0,0.049,NaN,-3.0,0.6,-2.4,-0.1,Sacramento Kings,"$4,588,384",Luc Mbah a Moute
8,2014-15,28.0,PHI,NBA,PF,67,61,28.6,3.7,9.5,...,1.1,0.027,NaN,-3.2,0.1,-3.2,-0.6,Philadelphia 76ers,"$4,382,576",Luc Mbah a Moute
9,2015-16,29.0,LAC,NBA,PF,75,61,17.0,1.3,2.8,...,1.6,0.059,NaN,-3.0,1.5,-1.6,0.1,Los Angeles Clippers,"$1,270,964",Luc Mbah a Moute
10,2016-17,30.0,LAC,NBA,SF,80,76,22.3,2.4,4.7,...,3.1,0.084,NaN,-0.8,1.6,0.7,1.2,Los Angeles Clippers,"$2,203,000",Luc Mbah a Moute


In [ ]:
https://www.basketball-reference.com/draft/NBA_2017.html